In [1]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

In [2]:
# Importamos Librerias
import numpy as np
import math
from collections import Counter
from sortedcontainers import SortedDict
import sklearn.naive_bayes as nb
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict

In [3]:
class Datos:

  TiposDeAtributos=('Continuo','Nominal')

  # TODO: procesar el fichero para asignar correctamente las variables tipoAtributos, nombreAtributos, nominalAtributos, datos y diccionarios
  # NOTA: No confundir TiposDeAtributos con tipoAtributos
  def __init__(self, nombreFichero):

      with open(nombreFichero, "r") as f:
        # Guardamos el numero de datos que contiene el DataSet y esta en la primera linea
        self.numDatos = int(f.readline())

        # Guardamos el nombre de los atributos
        self.nombreAtributos = f.readline().strip('\n').split(',')
        #print(self.nombreAtributos)

        # Leemos el tipo de los atributos de las variables y eliminamos el ultimo que es un salto de linea
        self.tipoAtributos = f.readline().strip('\n').split(',')
        #print(self.tipoAtributos)

        # Comprobamos que todos los atributos sean Continuos o Nominales
        if any(atr not in Datos.TiposDeAtributos for atr in self.tipoAtributos):
            raise ValueError("Tipo de atributo erroneo")

        # Segun el atributo, asignamos True o False.
        self.nominalAtributos = []

        # Guardamos en la lista nominalAtributos en la posicion de cada uno si es o no Nominal
        for tipo in self.tipoAtributos:
            if tipo == self.TiposDeAtributos[0]:
                self.nominalAtributos.append(False)
            else:
                self.nominalAtributos.append(True)
        #print(self.nominalAtributos)

        # Guardamos los datos del fichero y los formateamos, de tal forma que cada linea es una lista
        datos = f.readlines()
        datosFormat = []
        for lista in datos:
            datosFormat.append(lista.strip('\n').split(','))

        # print(set(sorted(datosFormat[0])))
        listaDatosAtributos = []
        for i in range(len(self.tipoAtributos)):
            listaDatosAtributos.append([])

        # Hacemos la traspuesta de los datos que guardamos para que cada lista de atributo guarde todos los datos
        # de cada atributo.
        for lista in datosFormat:
            i = 0
            for item in lista:
                listaDatosAtributos[i].append(item)
                i += 1

        # Ordenamos y hacemos un set para eliminar repetidos.
        i = 0
        for item in listaDatosAtributos:
            listaDatosAtributos[i] = sorted(set(item))
            i += 1


        # Creacion de lista diccionarios, en caso de que el atributo sea Continuo, el diccionario estara vacio
        self.listaDicts = []
        for i in range(len(self.tipoAtributos)):
            self.listaDicts.append({})

        # Creamos el diccionario tal y como se describe en las diapositivas, por orden y asignando valores numericos crecientes
        i = 0
        for atributo in listaDatosAtributos:
            k = 0
            if self.tipoAtributos[i] == "Nominal":
                for dato in atributo:
                    self.listaDicts[i][dato] = k
                    k += 1
            i += 1

        # Creacion de la matriz de datos utilizando el diccionario para mapear los valores
        # En primer lugar, creamos una matriz vacia de tamaña numero de atributos.
        self.datos = np.empty((int(self.numDatos),int(len(self.tipoAtributos))))
        i = 0
        j = 0

        # Metemos los datos en la matriz, mapeando con los diccionarios en el caso de que sean Nominales, y si son continuos normal.
        for i in range(int(self.numDatos)):
            for j in range(len(self.tipoAtributos)):
                if self.tipoAtributos[j] == 'Nominal':
                    self.datos[i][j] = self.listaDicts[j].get(str(datosFormat[i][j]))
                else:
                    self.datos[i][j] = datosFormat[i][j]
        
        print(self.nombreAtributos)
        print(self.listaDicts)
        f.close()


  # TODO: implementar en la practica 1
  def extraeDatos(self, idx):
    return self.datos[idx]


In [4]:
dataset = Datos('lenses.data')

print(dataset.datos)

['Age', 'Spectacle', 'Astigmatic', 'Tear', 'Class']
[{'1': 0, '2': 1, '3': 2}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1, '3': 2}]
[[0. 0. 0. 0. 2.]
 [0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 2.]
 [0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 2.]
 [0. 1. 0. 1. 1.]
 [0. 1. 1. 0. 2.]
 [0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 2.]
 [1. 0. 0. 1. 1.]
 [1. 0. 1. 0. 2.]
 [1. 0. 1. 1. 0.]
 [1. 1. 0. 0. 2.]
 [1. 1. 0. 1. 1.]
 [1. 1. 1. 0. 2.]
 [1. 1. 1. 1. 2.]
 [2. 0. 0. 0. 2.]
 [2. 0. 0. 1. 2.]
 [2. 0. 1. 0. 2.]
 [2. 0. 1. 1. 0.]
 [2. 1. 0. 0. 2.]
 [2. 1. 0. 1. 1.]
 [2. 1. 1. 0. 2.]
 [2. 1. 1. 1. 2.]]


In [5]:
from abc import ABCMeta,abstractmethod


class Particion():

  # Esta clase mantiene la lista de �ndices de Train y Test para cada partici�n del conjunto de particiones
    def __init__(self,train=[],test=[]):
        self.indicesTrain=train
        self.indicesTest=test
    
    def __str__(self):
        return "Train: {}\nTest:  {}".format(str(self.indicesTrain),str(self.indicesTest)) 

#####################################################################################################

class EstrategiaParticionado:

    # Clase abstracta
    __metaclass__ = ABCMeta

    # Lista de las particiones
    def __init__(self, nombre=""):
        self.nombreEstrategia = nombre
        self.numeroParticiones = 0
        self.particiones=[]

    # Atributos: deben rellenarse adecuadamente para cada estrategia concreta: nombreEstrategia, numeroParticiones, listaParticiones. Se pasan en el constructor

    @abstractmethod
    # TODO: esta funcion deben ser implementadas en cada estrategia concreta
    def creaParticiones(self,datos,seed=None):
        pass

In [6]:
class ValidacionSimple(EstrategiaParticionado):
    
    def __init__(self, porcentaje):
        self.porcentaje = porcentaje
        super().__init__("Validacion simple")
        
  # Crea particiones segun el metodo tradicional de division de los datos segun el porcentaje deseado.
  # Devuelve una lista de particiones (clase Particion)
  # TODO: implementar
    def creaParticiones(self,datos,seed=None):
        np.random.seed(seed)

        self.numeroParticiones = 1
    
        # Generamos una lista con todos los números de datos aleatorios   
        indicesAleatorios = np.random.permutation(int(datos.numDatos))
        
        # Creamos la particion, en funcion del porcentaje especificado
        self.particiones = [Particion(indicesAleatorios[:int(datos.numDatos*self.porcentaje)],
                                      indicesAleatorios[int(datos.numDatos*self.porcentaje):])]
        
        return self.particiones
    

In [7]:
validacion_simple = ValidacionSimple(0.75)
validacion_simple.creaParticiones(dataset)
print(validacion_simple.particiones[0])

Train: [10 23  2  6 20 18 11 16  1 15 12  4 13 17  5 19  7  3]
Test:  [21  0  9 22 14  8]


In [8]:
class ValidacionCruzada(EstrategiaParticionado):

  # Crea particiones segun el metodo de validacion cruzada.
  # El conjunto de entrenamiento se crea con las nfolds-1 particiones y el de test con la particion restante
  # Esta funcion devuelve una lista de particiones (clase Particion)
  # TODO: implementar
    
    def __init__(self, k):
        self.k = k
        super().__init__("Validacion cruzada")

    def creaParticiones(self,datos,seed=None):
        np.random.seed(seed)
        
        self.numeroParticiones = self.k
        
        # Generamos una lista con todos los números de datos aleatorios   
        indicesAleatorios = np.random.permutation(int(datos.numDatos))
        
        # Hallamos el tamaño de cada bloque
        tamBloque = int(datos.numDatos/self.k)
        
        
        datosSobran = datos.numDatos - (tamBloque*self.k)
        count = 0
        for i in range(self.k):
            
            train = np.delete(indicesAleatorios, range(i*tamBloque,(i+1)*tamBloque))
            test =  indicesAleatorios[i*tamBloque:(i+1)*tamBloque]
            
            # Caso en el que la cuenta es justa
            if datosSobran == 0:
                self.particiones.append(Particion(train, test))
                
            # Contemplamos el caso de que la division para sacar el numero de subconjuntos no fuese entera
            if datosSobran > 0:
                count += 1
                particionTest = np.append(test, train[(datos.numDatos - tamBloque)- i - 1])
                particionTrain = np.delete(train, (datos.numDatos - tamBloque)- i - 1)
                datosSobran -= 1
                self.particiones.append(Particion(particionTrain, particionTest))
                
            
           
                

In [9]:
v_cruzada = ValidacionCruzada(4)
v_cruzada.creaParticiones(dataset)

for particion in v_cruzada.particiones:
    print(particion)

Train: [ 1 13 22 10  0 20 21 17 16 11 14 23  2  4  3 19  7  9]
Test:  [ 8 15 18 12  6  5]
Train: [ 8 15 18 12  6  5 21 17 16 11 14 23  2  4  3 19  7  9]
Test:  [ 1 13 22 10  0 20]
Train: [ 8 15 18 12  6  5  1 13 22 10  0 20  2  4  3 19  7  9]
Test:  [21 17 16 11 14 23]
Train: [ 8 15 18 12  6  5  1 13 22 10  0 20 21 17 16 11 14 23]
Test:  [ 2  4  3 19  7  9]


In [10]:
class Clasificador:
  
    # Clase abstracta
    __metaclass__ = ABCMeta

    # Metodos abstractos que se implementan en casa clasificador concreto
    @abstractmethod
    # TODO: esta funcion debe ser implementada en cada clasificador concreto
    # datosTrain: matriz numpy con los datos de entrenamiento
    # atributosDiscretos: array bool con la indicatriz de los atributos nominales
    # diccionario: array de diccionarios de la estructura Datos utilizados para la codificacion de variables discretas
    def entrenamiento(self,datos,datosTrain,atributosDiscretos,diccionario):
        pass


    @abstractmethod
    # TODO: esta funcion debe ser implementada en cada clasificador concreto
    # devuelve un numpy array con las predicciones
    def clasifica(self,datosTest,atributosDiscretos,diccionario):
        pass


    # Obtiene el numero de aciertos y errores para calcular la tasa de fallo
    # TODO: implementar
    def error(self,datos,pred):
    # Aqui se compara la prediccion (pred) con las clases reales y se calcula el error    
        i = 0
        real = datos[:,-1]
        error = 0
        for i in range(len(real)):
            if real[i] != pred[i]:
                error += 1
        err = (error)/(len(real)+0.0)
        return err


    # Realiza una clasificacion utilizando una estrategia de particionado determinada
    # TODO: implementar esta funcion
    def validacion(self,particionado,dataset,clasificador,seed=None):

    # Creamos las particiones siguiendo la estrategia llamando a particionado.creaParticiones
    # - Para validacion cruzada: en el bucle hasta nv entrenamos el clasificador con la particion de train i
    # y obtenemos el error en la particion de test i
    # - Para validacion simple (hold-out): entrenamos el clasificador con la particion de train
    # y obtenemos el error en la particion test. Otra opci�n es repetir la validaci�n simple un n�mero especificado de veces, obteniendo en cada una un error. Finalmente se calcular�a la media.
        errores = 0
        #particionado.creaParticiones(dataset, seed)
    # Comprobamos si es por validación cruzada o simple, por la longitud de la lista de particiones
    # Validación Simple
        if len(particionado.particiones) == 1:
            clasificador.entrenamiento(dataset, particionado.particiones[0].indicesTrain)
            pred = clasificador.clasifica(dataset,particionado.particiones[0].indicesTest)
            ret = self.error(dataset.extraeDatos(particionado.particiones[0].indicesTest), pred)
            if ret > 0:
                return ret
            else:
                return 0
            
    # Validación Cruzada        
        else:
            for particion in particionado.particiones:
                clasificador.entrenamiento(dataset, particion.indicesTrain)
                pred = clasificador.clasifica(dataset,particion.indicesTest)
                ret = self.error(dataset.extraeDatos(particion.indicesTest), pred)
                errores += ret
            error = errores/len(particionado.particiones)
            #Devolucion de la media de los errores
            return error

In [34]:
class ClasificadorNaiveBayes(Clasificador):
    
    def __init__(self, laplace):
        self.laplace = laplace
        
    
    def entrenamiento(self,dataset,datosTrain):
     
        # Cargamos todos los datos de la clase del dataset desde la matriz de datos
        clasesTrain = dataset.extraeDatos(datosTrain)
        self.numClases = clasesTrain[:,-1]
        # Contamos las apariciones de cada uno para luego calcular la probabilidad a priori de cada clase
        counter = Counter(self.numClases)
        # Calculamos la probabilidad de la clase y lo metemos en un diccionario ordenado segun el numero 
        # correspondiente a cada clase asignado en el diccionario
        self.dictPrioris={}
        for k in counter:
            k = int(k)
            counter[k] = counter[k]/len(self.numClases)
            self.dictPrioris[k] = counter[k]
            
        # Aqui ordenamos el diccionario para que esten en el mismo orden de como extraemos los datos del dataset
        self.dictPrioris = SortedDict(self.dictPrioris)
        
        # Calcular tablas de probabilidades del entrenamiento. Tenemos que calcular por cada atributo una cuenta
        # de las apariciones en cada clase
        # Creamos una lista de matrices, donde vamos almacenar todos los datos que hemos obtenido en los datos de Test
        self.posteriori = np.zeros(len(dataset.nombreAtributos)-1,dtype=object)
        
        # Recorremos todos los datos de la matriz sin llegar a la clase
        for i in range(len(dataset.nombreAtributos) - 1):
            
            # Si el dato que obtenemos es Nominal haremos el recuento de todas las veces que sale la P(D|H)
            if dataset.nominalAtributos[i] == True:
                
                # Creamos una matriz de tamaño X: Número de Atributos menos la clase Y: Número de clases
                post = np.zeros((len(dataset.listaDicts[i]),len(dataset.listaDicts[-1])))
               
                # Aqui contamos todos las datos que queremos del datos Train para construir la matriz de entrenamiento
                for c in range(len(dataset.listaDicts[-1])):
                    datosEnt = dataset.extraeDatos(datosTrain)
                    dat = datosEnt[:,i]
                    repes = Counter(dat[datosEnt[:,-1] == c])
                    for r in repes:
                        post[int(r),c] = repes[r]
                    if self.laplace == True:
                        self.posteriori[i] = post + 1
                    else:
                        self.posteriori[i] = post
            
            # Si el dato es Continuo obtendremos la media y la desviación tipica de la clase
            else:
                
                # Creamos una matriz de X: Los datos de Media y Desivación típica Y: Número de clases
                post = np.zeros((2,len(dataset.listaDicts[-1])))
               
                # Aqui obtenemos la media y desviación tipica de cada clase, despues de tener los datos de entrenamiento
                for c in range(len(dataset.listaDicts[-1])):
                    datosEnt = dataset.extraeDatos(datosTrain)
                    dat = datosEnt[:,i]
                    datos = dat[datosEnt[:,-1] == c]
                    post[0][c] = np.mean(datos)
                    post[1][c] = np.std(datos)
                self.posteriori[i] = post
        
        #Calculamos los valores de los posteriori de todos las tablas anteriores
        for i in range(len(dataset.listaDicts) - 1):
            if dataset.nominalAtributos[i] == True:
                self.posteriori[i] /= sum(self.posteriori[i])
        
        print(self.posteriori)
        
    def clasifica(self,dataset,datosTest):
        j = 0
        aux = 1
        aux2 = 1
        self.prediccion = []
        datTest = dataset.extraeDatos(datosTest)
        #Ahora vamos a estudiar la probabilidad de la clase con los datos obtenidos en el entrenamiento
        #Recorremos todos las datos de la matriz de los datos Test
        
        for dato in datTest:
            mapa = []
            listaVerosimilitudes = []
            #Aqui obtenemos los prioris de cada clase para poder obtener la probabilidad de cada una
            for clase in range(len(self.dictPrioris)):
            #Aqui obtenemos cada valor posteriori de nuestro entrenamiento de los datos, es decir, P(D|H)
                for atributo in range(len(self.posteriori)):
                    if dataset.nominalAtributos[atributo] == True:
                        aux = self.posteriori[atributo][int(dato[atributo])][clase]
                        listaVerosimilitudes.append(aux)
                    #Aqui obtenemos la siguiente probabilidad P(D|H)*P(H)
                        #aux2 = self.dictPrioris.get(clase)*aux
                        #aux = 1
                
                    #Lo añadimos a una lista para obtener la probabilidad de las diferentes clases
                        #mapa.append(aux2)
            
            #Aqui obtenemos la probabilidad de los atibutos continuos
                    else:
                        # Hacemos la formula de la distribucion normal
                        exp1 = 1/(self.posteriori[atributo][1][clase]*math.sqrt(2*math.pi))
                        exp2 = np.power((dato[atributo]-self.posteriori[atributo][0][clase]) ,2)
                        exp3 = np.power(self.posteriori[atributo][1][clase],2)
                        exp4 = exp2/exp3   
                        exp4 = math.exp((-1/2)* exp3)
                        aux = exp1 * exp4
                        listaVerosimilitudes.append(aux)
                    #aux2 = aux * self.dictPrioris.get(clase)
                    for verosimilitud in listaVerosimilitudes:
                        aux2 *= verosimilitud
                    aux2 *= self.dictPrioris.get(clase)
                    mapa.append(aux2)
            
            #Aqui obtenemos la predicción de mayor probabilidad y la guardamos en nuestra lista de predicciones
            self.prediccion.append(np.argmax(mapa))
        
        #Devolvemos la lista con la predicción de nuestro clasifica   
        return self.prediccion

In [45]:
dataset = Datos('lenses.data')

validacion_simple = ValidacionSimple(0.75)
validacion_simple.creaParticiones(dataset)
v_cruzada = ValidacionCruzada(4)
v_cruzada.creaParticiones(dataset)

print("VALIDACION SIMPLE",validacion_simple.particiones[0])

print("VALIDACION CRUZADA")
for particion in v_cruzada.particiones:
    print(particion)


clasificador = ClasificadorNaiveBayes(True)
#clasificador.entrenamiento(dataset, validacion_simple.particiones[0].indicesTrain)
#clasificador.clasifica(dataset, validacion_simple.particiones[0].indicesTest)
error = clasificador.validacion(validacion_simple,dataset,clasificador)
error1 = clasificador.validacion(v_cruzada,dataset,clasificador)
print("El error del Clasificador NaiveBayes  para Validacion Simple es: "+str(error))
print("El error del Clasificador NaiveBayes para Validacion Cruzada es: "+str(error1))


['Age', 'Spectacle', 'Astigmatic', 'Tear', 'Class']
[{'1': 0, '2': 1, '3': 2}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1, '3': 2}]
VALIDACION SIMPLE Train: [ 2 14 18 20  0  5 16 23  9 13 19  8 21 17  4  7 12 22]
Test:  [11  1  6  3 10 15]
VALIDACION CRUZADA
Train: [11 12  6 23  1 15  9 19  3  2 21 16  0 17 20 22  8 18]
Test:  [ 5 13 10  4  7 14]
Train: [ 5 13 10  4  7 14  9 19  3  2 21 16  0 17 20 22  8 18]
Test:  [11 12  6 23  1 15]
Train: [ 5 13 10  4  7 14 11 12  6 23  1 15  0 17 20 22  8 18]
Test:  [ 9 19  3  2 21 16]
Train: [ 5 13 10  4  7 14 11 12  6 23  1 15  9 19  3  2 21 16]
Test:  [ 0 17 20 22  8 18]
[array([[0.4       , 0.28571429, 0.26666667],
       [0.2       , 0.42857143, 0.26666667],
       [0.4       , 0.28571429, 0.46666667]])
 array([[0.5       , 0.33333333, 0.5       ],
       [0.5       , 0.66666667, 0.5       ]])
 array([[0.25      , 0.83333333, 0.57142857],
       [0.75      , 0.16666667, 0.42857143]])
 array([[0.25      , 0.16666667, 

In [36]:
dataset = Datos('tic-tac-toe.data')
print(dataset.datos)

['TLeftSq', 'TMidSq', 'TRightSq', 'MLeftSq', 'MMidSq', 'MRightSq', 'BLeftSq', 'BMidSq', 'BRightSq', 'Class']
[{'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'negative': 0, 'positive': 1}]
[[2. 2. 2. ... 1. 1. 1.]
 [2. 2. 2. ... 2. 1. 1.]
 [2. 2. 2. ... 1. 2. 1.]
 ...
 [1. 2. 1. ... 1. 2. 0.]
 [1. 2. 1. ... 1. 2. 0.]
 [1. 1. 2. ... 2. 2. 0.]]


In [37]:
validacion_simple = ValidacionSimple(0.75)
validacion_simple.creaParticiones(dataset)
v_cruzada = ValidacionCruzada(4)
v_cruzada.creaParticiones(dataset)

print("VALIDACION SIMPLE",validacion_simple.particiones[0])

print("VALIDACION CRUZADA")
for particion in v_cruzada.particiones:
    print(particion)


VALIDACION SIMPLE Train: [783 490 950 735 855 114 360 741 286 555 179 296 109 193 612 519 800 229
 849 771 891 533 649 327 790 207 657 576 408 161   2 453 271 278 734  96
 943 136 556 169 395 619 727 369 819 331 303 941 374 905 754 291 840 319
  33 643 957 514 219 564 701  99 144 546  63 908 377 353 164 486 656 324
 895 744 842 104 538 308 607 347 380 216  72 390 893  91 357 696 591 530
  79 138 631 238 798 243 833 854 641 378 304 223  94 251  15 312 700 737
 416  31 860  80 276 823 863 338 156 418 780 911 392 715 723 261 586 232
 439 579 677 588 826 709 717 875 901 874 181 242 567 830 410 436 236 603
 926 496 367 589 165 124 337  17  69 195  46 764 542 153 821 774 134 608
 801 836 180 906 454 518 267 280 651 379 802 192 786 565 814 756 864 472
 655 503 550 174 932 406  13 707 433 215 295 497  74 585 539 610 147 102
 817 168  41 113 545 264 345 523 729 832 103 279 277 812 805 557 804 307
 535 465 351 776 653  90 116 675 464 505 362 865 639 697 676  93 450 435
 182 809 558 910 660 140 4

In [38]:
clasificador = ClasificadorNaiveBayes(True)
#clasificador.entrenamiento(dataset, validacion_simple.particiones[0].indicesTrain)
#clasificador.clasifica(dataset, validacion_simple.particiones[0].indicesTest)
error = clasificador.validacion(validacion_simple,dataset,clasificador)
error1 = clasificador.validacion(v_cruzada,dataset,clasificador)
print("El error del Clasificador NaiveBayes  para Validacion Simple es: "+str(error))
print("El error del Clasificador NaiveBayes para Validacion Cruzada es: "+str(error1))

[array([[0.18677043, 0.22912206],
       [0.45136187, 0.29336188],
       [0.3618677 , 0.47751606]])
 array([[0.21400778, 0.26766595],
       [0.31128405, 0.38543897],
       [0.47470817, 0.34689507]])
 array([[0.19066148, 0.25695931],
       [0.45136187, 0.29122056],
       [0.35797665, 0.45182013]])
 array([[0.24124514, 0.25481799],
       [0.30350195, 0.35974304],
       [0.45525292, 0.38543897]])
 array([[0.13229572, 0.17344754],
       [0.57198444, 0.21627409],
       [0.29571984, 0.61027837]])
 array([[0.24124514, 0.26124197],
       [0.30350195, 0.37901499],
       [0.45525292, 0.35974304]])
 array([[0.19455253, 0.21627409],
       [0.41245136, 0.32334047],
       [0.39299611, 0.46038544]])
 array([[0.23346304, 0.27837259],
       [0.30350195, 0.36616702],
       [0.46303502, 0.35546039]])
 array([[0.19844358, 0.23126338],
       [0.44747082, 0.30406852],
       [0.3540856 , 0.46466809]])]
[array([[0.19305019, 0.24731183],
       [0.46332046, 0.31397849],
       [0.34362934, 0.4

In [39]:
dataset = Datos('german.data')
print(dataset.datos)
print(dataset.nominalAtributos)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'Class']
[{'A11': 0, 'A12': 1, 'A13': 2, 'A14': 3}, {}, {'A30': 0, 'A31': 1, 'A32': 2, 'A33': 3, 'A34': 4}, {'A40': 0, 'A41': 1, 'A410': 2, 'A42': 3, 'A43': 4, 'A44': 5, 'A45': 6, 'A46': 7, 'A48': 8, 'A49': 9}, {}, {'A61': 0, 'A62': 1, 'A63': 2, 'A64': 3, 'A65': 4}, {'A71': 0, 'A72': 1, 'A73': 2, 'A74': 3, 'A75': 4}, {}, {'A91': 0, 'A92': 1, 'A93': 2, 'A94': 3}, {'A101': 0, 'A102': 1, 'A103': 2}, {}, {'A121': 0, 'A122': 1, 'A123': 2, 'A124': 3}, {}, {'A141': 0, 'A142': 1, 'A143': 2}, {'A151': 0, 'A152': 1, 'A153': 2}, {}, {'A171': 0, 'A172': 1, 'A173': 2, 'A174': 3}, {}, {'A191': 0, 'A192': 1}, {'A201': 0, 'A202': 1}, {'1': 0, '2': 1}]
[[ 0.  6.  4. ...  1.  0.  0.]
 [ 1. 48.  2. ...  0.  0.  1.]
 [ 3. 12.  4. ...  0.  0.  0.]
 ...
 [ 3. 12.  2. ...  0.  0.  0.]
 [ 0. 45.  2. ...  1.  0.  1.]
 [ 1. 45.  4. ...  0.  0.  0.]]
[True, False, True, True, False,

In [40]:
validacion_simple = ValidacionSimple(0.75)
validacion_simple.creaParticiones(dataset)
v_cruzada = ValidacionCruzada(4)
v_cruzada.creaParticiones(dataset)

print("VALIDACION SIMPLE",validacion_simple.particiones[0])

print("VALIDACION CRUZADA")
for particion in v_cruzada.particiones:
    print(particion)

VALIDACION SIMPLE Train: [ 10 658 191 526 580 275 115 883 313 917 657 901 956 147 167  94  59 927
 626 994 300 183 716  69   6 798  80 815 918   7 232 975 863 397 969 527
  27 260 203 700 490 573 635 385 367 129 229 306  16 641 957 494 495 981
 244 923  82 955 190 760 805 324 225 731 705 179 396 356 139 492 180 600
 800 920 717 898 320 733 607 553 256 997 157 382 652 277 144 655 727 565
 473 257 908 867 561  65 888 948 255 807 538 638 295 298 154 519 146 413
 905 667 811 511 673 804 741 695 732 629 419 281 343 149 390 634 389  25
 756 265 884 982   8 259  84 323 207 424 719 162 690  53 909 269 465 577
 590 108  73 394 215 168 656 886 639 833 770 400 684   9 570 120 329 744
 556 808 375   4 430 458 572  97 462 788 118 537 161 876 686 192  88 661
 204 819 582 848 604 765 464 959 976 757  40 483 498 562 979 958 568 407
 502 858 640 581 794 476 872 383 718  95 401 294 485 373 970 500 859 615
 405 467 775 282  51 211 826 218 348 967 188 643  15 431 738 292 377 993
 450 688 772 230 664 482 2

In [41]:
clasificador = ClasificadorNaiveBayes(True)
#clasificador.entrenamiento(dataset, validacion_simple.particiones[0].indicesTrain)
#clasificador.clasifica(dataset, validacion_simple.particiones[0].indicesTest)
error = clasificador.validacion(validacion_simple,dataset,clasificador)
error1 = clasificador.validacion(v_cruzada,dataset,clasificador)
print("El error del Clasificador NaiveBayes  para Validacion Simple es: "+str(error))
print("El error del Clasificador NaiveBayes para Validacion Cruzada es: "+str(error1))

[array([[0.20260223, 0.44545455],
       [0.25092937, 0.32727273],
       [0.06877323, 0.05909091],
       [0.47769517, 0.16818182]])
 array([[19.24906367, 24.5       ],
       [10.95854245, 13.27277916]])
 array([[0.02411874, 0.08144796],
       [0.03339518, 0.09049774],
       [0.50278293, 0.55656109],
       [0.08348794, 0.09954751],
       [0.35621521, 0.1719457 ]])
 array([[0.19852941, 0.30530973],
       [0.12132353, 0.04867257],
       [0.01102941, 0.02212389],
       [0.17830882, 0.18141593],
       [0.30882353, 0.22123894],
       [0.01470588, 0.01769912],
       [0.02389706, 0.03539823],
       [0.04411765, 0.05309735],
       [0.01654412, 0.00884956],
       [0.08272059, 0.10619469]])
 array([[2949.76217228, 3803.81018519],
       [2346.83292876, 3496.08345233]])
 array([[0.52875696, 0.70588235],
       [0.11131725, 0.12217195],
       [0.0742115 , 0.04072398],
       [0.06493506, 0.02714932],
       [0.22077922, 0.1040724 ]])
 array([[0.05751391, 0.09049774],
       [0.1447

In [42]:
#SKLearn Validacion Cruzada
# Hacemos el encode de los datos
encAtributos = preprocessing.OneHotEncoder(categorical_features=dataset.nominalAtributos[:-1],sparse=False)
X = encAtributos.fit_transform(dataset.datos[:,:-1])
Y = dataset.datos[:,-1]

# Partimos los datos en Train y Test
x_train, x_test, y_train,y_test = train_test_split(X,Y,test_size=0.2)

# Aplicamos Naive Bayes sin Laplace
clf = MultinomialNB(alpha = 0,fit_prior=True, class_prior=None)
predicciones=clf.fit(x_train,y_train).predict(x_test)

# Print de los resultados
print("Predicciones utilizando validacion simple sin correcion de Laplace:", predicciones)
print("Clases reales de la particion de text", y_test)

# Aplicamos Naive Bayes con Laplace
clf = MultinomialNB(alpha = 0,fit_prior=True, class_prior=None)
predicciones=clf.fit(x_train,y_train).predict(x_test)

# Print de los resultados
print("Predicciones utilizando validacion simple con correcion de Laplace:", predicciones)
print("Clases reales de la particion de text", y_test)



Predicciones utilizando validacion simple sin correcion de Laplace: [1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1.
 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1.
 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 1. 1. 0. 1. 1. 0. 0. 0.]
Clases reales de la particion de text [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1.
 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1.
 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0

/home/pablo/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/pablo/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/home/pablo/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too s

In [20]:
# SKLearn Validacion Cruzada
# Hacemos el encode de los datos
encAtributos = preprocessing.OneHotEncoder(categorical_features=dataset.nominalAtributos[:-1],sparse=False)
X = encAtributos.fit_transform(dataset.datos[:,:-1])
Y = dataset.datos[:,-1]

# Partimos los datos en Train y Test
x_train, x_test, y_train,y_test = train_test_split(X,Y,test_size=0.2)

# Aplicamos Naive Bayes sin Laplace
clf = MultinomialNB(alpha = 0,fit_prior=True, class_prior=None)
predicciones=clf.fit(x_train,y_train).predict(x_test)

# Print de los resultados
print("Predicciones utilizando validacion simple sin correcion de Laplace:", predicciones)
print("Clases reales de la particion de text", y_test)

# Aplicamos Naive Bayes con Laplace
clf = MultinomialNB(alpha = 0,fit_prior=True, class_prior=None)
predicciones=clf.fit(x_train,y_train).predict(x_test)

# Print de los resultados
print("Predicciones utilizando validacion simple con correcion de Laplace:", predicciones)
print("Clases reales de la particion de text", y_test)


Predicciones utilizando validacion simple sin correcion de Laplace: [0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1.
 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 0. 1. 1.]
Clases reales de la particion de text [0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0

/home/pablo/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/pablo/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/home/pablo/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too s